# LLM Model Comparison using OpenRouter Rankings

This notebook evaluates and compares the top language models from [OpenRouter Rankings](https://openrouter.ai/rankings).

## What This Notebook Does

1. **Fetches Top Models** - Scrapes the current rankings from OpenRouter to identify the most popular models
2. **Generates Questions** - Each model creates a challenging reasoning question
3. **Answers Questions** - Each model answers all questions from other models
4. **Evaluates Responses** - Each model rates the quality of all answers on a 10-point scale
5. **Aggregates Results** - Produces comparison metrics and cross-model rating matrices

## Requirements

- **OpenRouter API Key**: Set the `OPENROUTER_API_KEY` environment variable
- **Dependencies**: pandas, requests, beautifulsoup4, playwright, openai (automatically installed with `uv sync`)

In [ ]:
# Core imports for the notebook
import os  # Environment variable access
from typing import Any  # Type hints

# Data manipulation and analysis
import pandas as pd  # DataFrames for structured data

# Web scraping and API calls
import requests  # HTTP requests for OpenRouter API
from bs4 import BeautifulSoup  # HTML parsing for rankings page

# Note: Additional imports (re, time, playwright, openai) are imported
# within specific cells where they're needed

## Evaluation Pipeline Flow

Here's how the evaluation works step-by-step:

```
┌─────────────────────────────────────────────────────────┐
│  Step 1: Fetch Top Models from OpenRouter Rankings      │
│  → Scrapes live usage data OR uses API sorting          │
└─────────────────┬───────────────────────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────────────────────┐
│  Step 2: Question Generation (N models)                 │
│  → Each model creates 1 challenging question            │
│  → Total: N questions                                   │
└─────────────────┬───────────────────────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────────────────────┐
│  Step 3: Answer Generation (N × N combinations)         │
│  → Each model answers every question                    │
│  → Total: N × N answers                                 │
└─────────────────┬───────────────────────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────────────────────┐
│  Step 4: Answer Evaluation (N × N x N ratings)          │
│  → Each model rates every answer (1-10 scale)           │
│  → Total: N × N x N ratings                             │
└─────────────────┬───────────────────────────────────────┘
                  │
                  ▼
┌─────────────────────────────────────────────────────────┐
│  Step 5: Aggregate Results                              │
│  → Average ratings per model                            │
│  → Cross-model rating matrix                            │
│  → Performance insights                                 │
└─────────────────────────────────────────────────────────┘
```

**Example with 5 models:**
- 5 questions generated
- 25 answers generated (5 models × 5 questions)
- 125 ratings collected (5 models × 5 answers x 5 questions)
- Total LLM calls: 155

## Setup and Configuration

Before running this notebook, you need to:

1. **Install Playwright browsers** (one-time setup):
```bash
uv run playwright install chromium
```

2. **Set your OpenRouter API Key**:
   - Get a free API key from [OpenRouter](https://openrouter.ai/)
   - Set it as an environment variable:
     - **Windows (PowerShell)**: `$env:OPENROUTER_API_KEY="your-key-here"`
     - **Mac/Linux**: `export OPENROUTER_API_KEY="your-key-here"`
   - Or add it to a `.env` file in the project root

The notebook will check for this API key before making requests.

## Fetch Top Models from OpenRouter

This section scrapes the OpenRouter rankings page to get real-time popularity data based on actual usage.

### How It Works

The `fetch_openrouter_rankings()` function uses **Playwright** (a browser automation tool) to:
1. Launch a headless browser
2. Navigate to the OpenRouter rankings page
3. Wait for JavaScript content to load
4. Extract model data including rank, name, token usage, and usage trends

**Why Playwright?** The rankings page uses JavaScript to render content dynamically, so we need a real browser to see the data.

**Windows Event Loop Note:** Jupyter uses an asyncio event loop that conflicts with Playwright on Windows. The function runs Playwright in a separate thread with its own event loop to avoid this issue.

### Available Sorting Options

The `fetch_top_openrouter_models()` function supports different sorting criteria:
- **popularity** - Models ranked by actual usage on OpenRouter (default, uses Playwright scraping)
- **price_low** - Cheapest models first
- **price_high** - Most expensive (often most capable) models first  
- **context_length** - Longest context window first
- **newest** - Most recently added models first

**Note:** Only the popularity ranking requires Playwright. Other sorting options use the OpenRouter API directly.

In [ ]:
# Fallback model list used when API calls or web scraping fails
FALLBACK_MODELS = [
    {
        "id": "anthropic/claude-3.5-sonnet",
        "name": "Claude 3.5 Sonnet",
        "description": "Anthropic Claude 3.5 Sonnet",
        "context_length": 200000,
    },
    {
        "id": "openai/gpt-4o",
        "name": "GPT-4o",
        "description": "OpenAI GPT-4o",
        "context_length": 128000,
    },
    {
        "id": "google/gemini-pro-1.5",
        "name": "Gemini Pro 1.5",
        "description": "Google Gemini Pro 1.5",
        "context_length": 1_000_000,
    },
    {
        "id": "meta-llama/llama-3.1-405b-instruct",
        "name": "Llama 3.1 405B",
        "description": "Meta Llama 3.1 405B Instruct",
        "context_length": 128000,
    },
    {
        "id": "anthropic/claude-3.5-haiku",
        "name": "Claude 3.5 Haiku",
        "description": "Anthropic Claude 3.5 Haiku",
        "context_length": 100000,
    },
]

In [ ]:
def fetch_openrouter_rankings() -> pd.DataFrame:
    """
    Scrape the OpenRouter rankings page to get the current top models by actual usage.
    Uses Playwright async API run in a separate thread to avoid event loop conflicts.

    Returns:
        DataFrame with columns: rank, model_id, model_name, tokens, token_change
    """
    import asyncio
    import concurrent.futures
    import sys

    async def _async_fetch():
        """Internal async function to fetch rankings using browser automation"""
        try:
            import re

            from playwright.async_api import async_playwright

            print("Launching browser to fetch rankings...")

            async with async_playwright() as p:
                # Launch browser in headless mode (no visible window)
                browser = await p.chromium.launch(headless=True)
                page = await browser.new_page()

                try:
                    # Navigate to rankings page with short timeout (user can retry if it fails)
                    await page.goto(
                        "https://openrouter.ai/rankings",
                        wait_until="domcontentloaded",
                        timeout=15000,
                    )

                    # Wait for the leaderboard section to appear in the DOM
                    await page.wait_for_selector("#leaderboard", timeout=10000, state="attached")

                    # Give JavaScript time to populate the content (5 seconds)
                    await page.wait_for_timeout(5000)

                    # Get the fully rendered HTML after JavaScript execution
                    html = await page.content()

                finally:
                    await browser.close()

            # Parse the rendered HTML with BeautifulSoup
            soup = BeautifulSoup(html, "html.parser")
            leaderboard = soup.find(id="leaderboard")

            if not leaderboard:
                print("⚠ Leaderboard section not found in rendered page")
                return pd.DataFrame(columns=["rank", "model_id", "model_name", "tokens", "token_change"])

            # Extract model information using CSS selectors
            rankings_data: list[dict[str, object]] = []

            # Find all leaderboard entries (each entry is a grid container with 12 columns)
            entries = leaderboard.select("div.grid.grid-cols-12.items-center")

            for entry in entries[:30]:  # Limit to top 30 models
                try:
                    # Column 1: Extract rank number (e.g., "1.", "2.")
                    rank_elem = entry.select_one("div.col-span-1")
                    rank = int(rank_elem.get_text(strip=True).replace(".", "")) if rank_elem else None

                    # Column 2: Extract model name and ID from the link
                    model_link = entry.select_one("div.col-span-7 a.font-medium")
                    if not model_link:
                        continue

                    model_name = model_link.get_text(strip=True)
                    href = model_link.get("href", "")
                    # Remove leading "/" from href to get model_id
                    model_id = href[1:] if isinstance(href, str) and href.startswith("/") else href

                    # Column 3: Extract token count and change percentage
                    tokens: int | None = None
                    token_change: str | None = None
                    token_container = entry.select_one("div.col-span-4")

                    if token_container:
                        divs = token_container.select("div")

                        # First div contains the token count (e.g., "1.04T tokens", "801B tokens")
                        if divs:
                            token_text = divs[0].get_text(strip=True)
                            # Parse token count with units (K=thousand, M=million, B=billion, T=trillion)
                            token_match = re.search(r"([\d.]+)([KMBT])\s*tokens", token_text, re.IGNORECASE)
                            if token_match:
                                value = float(token_match.group(1))
                                unit = token_match.group(2).upper()
                                multipliers = {
                                    "K": 1_000,
                                    "M": 1_000_000,
                                    "B": 1_000_000_000,
                                    "T": 1_000_000_000_000,
                                }
                                tokens = int(value * multipliers.get(unit, 1))

                        # Extract percentage change (usage trend)
                        percent_div = token_container.select_one("div.mt-1")
                        if percent_div:
                            svg_elem = percent_div.select_one("svg")
                            full_text = percent_div.get_text(strip=True)
                            percent_match = re.search(r"([\d.]+)%", full_text)
                            if percent_match:
                                percentage_value = percent_match.group(1)
                                svg_class = ""
                                if svg_elem:
                                    svg_class_raw = svg_elem.get("class", [])
                                    if isinstance(svg_class_raw, list):
                                        svg_class = " ".join(svg_class_raw)
                                    else:  # str
                                        svg_class = str(svg_class_raw)
                                # Red SVG indicates decrease, green indicates increase
                                if svg_class and "text-red" in svg_class:
                                    token_change = f"-{percentage_value}%"
                                else:
                                    token_change = f"{percentage_value}%"

                    rankings_data.append(
                        {
                            "rank": rank,
                            "model_id": model_id,
                            "model_name": model_name,
                            "tokens": tokens,
                            "token_change": token_change,
                        }
                    )

                except (ValueError, AttributeError):
                    # Skip entries that don't match expected format
                    continue

            if rankings_data:
                print(f"✓ Successfully extracted {len(rankings_data)} models from rankings page")
                return pd.DataFrame(rankings_data)

            print("⚠ No models found, using fallback")
            raise ValueError("No models extracted")

        except Exception as err:  # noqa: BLE001
            print(f"Error with Playwright: {err}")
            import traceback

            traceback.print_exc()
            raise

    def _run_in_thread():
        """
        Run async function in a new event loop in a separate thread.
        This avoids conflicts with Jupyter's event loop on Windows.
        """
        # On Windows, use ProactorEventLoop which supports subprocesses
        if sys.platform == "win32":
            loop = asyncio.WindowsProactorEventLoopPolicy().new_event_loop()
        else:
            loop = asyncio.new_event_loop()

        asyncio.set_event_loop(loop)
        try:
            return loop.run_until_complete(_async_fetch())
        finally:
            loop.close()

    try:
        # Run in a thread pool to avoid event loop conflicts with Jupyter
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(_run_in_thread)
            return future.result(timeout=30)

    except Exception as err:  # noqa: BLE001
        print(f"Error fetching rankings: {err}")

        # Fallback to hardcoded list if scraping fails
        print("Using fallback: hardcoded top models list")
        fallback_rankings = [
            {
                "rank": i + 1,
                "model_id": model["id"],
                "model_name": model["name"],
                "tokens": None,
                "token_change": None,
            }
            for i, model in enumerate(FALLBACK_MODELS)
        ]
        return pd.DataFrame(fallback_rankings)


def fetch_top_openrouter_models(top_n: int = 5, sort_by: str = "popularity") -> list[dict]:
    """
    Fetch the top N models from OpenRouter using various sorting criteria.

    Args:
        top_n: Number of top models to return
        sort_by: Sorting criterion - 'popularity', 'price_low', 'price_high',
                'context_length', 'newest'

    Returns:
        List of dictionaries containing model information with keys:
        id, name, description, context_length, pricing, created, avg_cost
    """
    url = "https://openrouter.ai/api/v1/models"

    try:
        response = requests.get(url, timeout=30)
        response.raise_for_status()
        data = response.json()

        # Extract model data from API response
        models = data.get("data", [])

        # Filter out models without proper pricing or context info
        valid_models: list[dict[str, object]] = []
        for model in models:
            pricing = model.get("pricing", {})
            # Only include models with valid pricing information
            if pricing and pricing.get("prompt") and pricing.get("completion"):
                try:
                    prompt_cost = float(pricing.get("prompt", "0"))
                    completion_cost = float(pricing.get("completion", "0"))
                except (TypeError, ValueError):
                    continue
                model_info = {
                    "id": model.get("id", ""),
                    "name": model.get("name", model.get("model_id", "")),
                    "description": model.get("description", "No description available"),
                    "context_length": model.get("context_length", 0),
                    "pricing": pricing,
                    "created": model.get("created", 0),
                    # Calculate average cost per 1M tokens for easy comparison
                    "avg_cost": (prompt_cost + completion_cost) / 2 * 1_000_000,
                }
                valid_models.append(model_info)

        # Sort models based on the specified criterion
        if sort_by == "popularity":
            print("Fetching current model rankings from OpenRouter (using Playwright)...")
            rankings_df = fetch_openrouter_rankings()
            if rankings_df is not None and not rankings_df.empty:
                # Create a mapping of model_id to rank
                popularity_order = {row["model_id"]: row["rank"] for _, row in rankings_df.iterrows()}
                # Sort by rank (lower rank = more popular), default to 999 for unranked
                sorted_models = sorted(valid_models, key=lambda x: popularity_order.get(x["id"], 999))
                print(f"✓ Successfully ranked {len(rankings_df)} models by current usage data")
            else:
                print("⚠ Could not fetch rankings, using default sort")
                sorted_models = valid_models
        elif sort_by == "price_low":
            sorted_models = sorted(valid_models, key=lambda x: x["avg_cost"])
        elif sort_by == "price_high":
            sorted_models = sorted(valid_models, key=lambda x: x["avg_cost"], reverse=True)
        elif sort_by == "context_length":
            sorted_models = sorted(valid_models, key=lambda x: x["context_length"], reverse=True)
        elif sort_by == "newest":
            sorted_models = sorted(valid_models, key=lambda x: x["created"], reverse=True)
        else:
            sorted_models = valid_models

        return sorted_models[:top_n]

    except Exception as err:  # noqa: BLE001
        print(f"Error fetching models: {err}")
        # Fallback to a hardcoded list of popular models
        return [{**model, "pricing": {}, "avg_cost": 0} for model in FALLBACK_MODELS][:top_n]

In [ ]:
# Test: Fetch and display the current rankings
rankings_df = fetch_openrouter_rankings()

print(f"Successfully fetched {len(rankings_df)} ranked models\n")
print("Top 10 Models by Usage on OpenRouter:")
print("=" * 80)

display_df = rankings_df.head(10).copy()


def format_tokens(tokens: int | None) -> str:
    if tokens is None:
        return "N/A"
    if tokens >= 1_000_000_000_000:
        return f"{tokens / 1_000_000_000_000:.2f}T"
    if tokens >= 1_000_000_000:
        return f"{tokens / 1_000_000_000:.1f}B"
    if tokens >= 1_000_000:
        return f"{tokens / 1_000_000:.1f}M"
    return f"{tokens:,}"


display_df["tokens_formatted"] = display_df["tokens"].apply(format_tokens)
display_df[["rank", "model_name", "model_id", "tokens_formatted", "token_change"]]

### Understanding the Rankings Output

The table above shows:

- **rank**: Current position on OpenRouter (lower = more popular)
- **model_name**: Human-readable name of the model
- **model_id**: Unique identifier used for API calls (format: `provider/model-name`)
- **tokens_formatted**: Total tokens processed (T=trillion, B=billion, M=million)
- **token_change**: Usage trend as percentage change (↑ green positive, ↓ red negative)

**What does this tell us?**
Models at the top are being used most heavily by real users on OpenRouter, which often (but not always) correlates with quality, speed, or value. This ranking updates in real-time based on actual API usage.

In [ ]:
# Remove ":free" suffix from model IDs for cleaner display
# OpenRouter sometimes appends ":free" to free-tier models
display_df["model_id"] = display_df["model_id"].str.replace(r":free$", "", regex=True)

In [ ]:
# Select top 5 models from the rankings for our evaluation
# Alternative approach (commented out): fetch via API with different sorting
# top_models = fetch_top_openrouter_models(5, sort_by="popularity")
# df_models = pd.DataFrame(top_models)

top_models = display_df.head(5)

In [ ]:
# Validate that we have the required data for the evaluation
print("Data Validation:")
print("=" * 60)

# Check we have models
if len(display_df) < 3:
    print("⚠️  Warning: Less than 3 models available")
    print("   Results may be less meaningful with fewer models")
else:
    print(f"✓ {len(display_df)} models available for evaluation")

# Check for required columns
required_cols = ["model_id", "model_name"]
missing_cols = [col for col in required_cols if col not in display_df.columns]
if missing_cols:
    print(f"❌ Missing required columns: {missing_cols}")
else:
    print(f"✓ All required columns present: {required_cols}")

# Check for duplicate models
duplicates = display_df["model_id"].duplicated().sum()
if duplicates > 0:
    print(f"⚠️  Warning: {duplicates} duplicate model IDs found")
else:
    print("✓ No duplicate models")

print("\nReady to proceed with evaluation! 🚀")

In [ ]:
top_models

In [ ]:
# Configure the OpenRouter API client
# OpenRouter uses an OpenAI-compatible API, so we use the OpenAI Python client
from openai import OpenAI

# Get API key from environment variable
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

if not OPENROUTER_API_KEY:
    error_msg = """
    ❌ OPENROUTER_API_KEY not found!
    
    To fix this issue:
    
    1. Get a free API key from: https://openrouter.ai/
       (Sign up with GitHub or email)
    
    2. Set the environment variable:
       
       Windows PowerShell:
         $env:OPENROUTER_API_KEY="sk-or-v1-xxxxx"
       
       Windows CMD:
         set OPENROUTER_API_KEY=sk-or-v1-xxxxx
       
       Mac/Linux:
         export OPENROUTER_API_KEY="sk-or-v1-xxxxx"
    
    3. Restart this notebook kernel (Kernel → Restart)
    
    Alternative: Create a .env file in the project root:
       OPENROUTER_API_KEY=sk-or-v1-xxxxx
    """
    raise ValueError(error_msg)

# Validate API key format
if not OPENROUTER_API_KEY.startswith("sk-or-"):
    print("⚠️  Warning: API key doesn't start with 'sk-or-'")
    print("   This might not be a valid OpenRouter API key")
    print("   Expected format: sk-or-v1-xxxxx")

# Create client with OpenRouter's base URL
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=OPENROUTER_API_KEY)

print("✓ OpenRouter client configured successfully!")
print(f"  API key: {OPENROUTER_API_KEY[:15]}...{OPENROUTER_API_KEY[-4:]}")
print("  Base URL: https://openrouter.ai/api/v1")

## Model Comparison Pipeline

The evaluation consists of three phases:

1. **Question Generation** - Each model creates one challenging reasoning question
2. **Answer Generation** - Each model answers every question from all other models  
3. **Answer Evaluation** - Each model rates every answer on a 10-point scale

This creates a comprehensive cross-comparison where models evaluate each other's performance.

In [ ]:
import time

# Display how many models we're using for the evaluation pipeline
num_models = len(top_models)
total_questions = num_models
total_answers = num_models * num_models
total_ratings = num_models * num_models * total_questions
total_api_calls = total_questions + total_answers + total_ratings

print(f"Using {num_models} models for the evaluation pipeline:")
print("  - Each model will generate 1 question")
print(f"  - Each model will answer {num_models} questions")
print(f"  - Each model will rate {total_answers} answers")
print(f"\nTotal API calls: {total_api_calls}")
print(f"  • {total_questions} question generation calls")
print(f"  • {total_answers} answer generation calls")
print(f"  • {total_ratings} rating calls")

# Estimate time based on typical API response times
avg_question_time = 3  # seconds
avg_answer_time = 5  # seconds
avg_rating_time = 2  # seconds

estimated_time = total_questions * avg_question_time + total_answers * avg_answer_time + total_ratings * avg_rating_time

print(f"\n⏱️  Estimated total time: ~{estimated_time // 60} minutes {estimated_time % 60} seconds")
print(f"   (assuming {avg_question_time}s/question, {avg_answer_time}s/answer, {avg_rating_time}s/rating)")
print("\n💡 Tip: Actual time may vary based on model speed and API load")

In [ ]:
def generate_cost_summary_markdown(summary_df:pd.DataFrame) -> str:
    """
    Generate a markdown-formatted cost summary from the summary DataFrame.

    Args:
        summary_df: DataFrame with cost and token usage data

    Returns:
        String containing markdown-formatted cost summary
    """
    if summary_df.empty:
        return "⚠️ No token data found in any DataFrames\n"

    markdown_lines = []

    # Header
    markdown_lines.append("## 📊 TOKEN USAGE & COST SUMMARY (ACTUAL COSTS)")
    markdown_lines.append("")

    # Overall totals
    total_input_tokens = summary_df["input_tokens"].sum()
    total_output_tokens = summary_df["output_tokens"].sum()
    total_tokens = summary_df["total_tokens"].sum()
    total_time = round(summary_df["time_s"].sum(), 2)
    total_cost = summary_df["actual_cost_usd"].sum()
    successful_fetches = summary_df["cost_fetch_success"].sum()
    total_calls = len(summary_df)

    markdown_lines.append("## 🔢 OVERALL TOTALS")
    markdown_lines.append("")
    markdown_lines.append(f"- **Input tokens:** {total_input_tokens:,}")
    markdown_lines.append(f"- **Output tokens:** {total_output_tokens:,}")
    markdown_lines.append(f"- **Total tokens:** {total_tokens:,}")
    markdown_lines.append(f"- **Total time (s):** {total_time:,}")
    markdown_lines.append(f"- **Actual cost:** ${total_cost:.2f}")
    markdown_lines.append(
        f"- **Cost data success:** {successful_fetches}/{total_calls} calls ({successful_fetches/total_calls*100:.1f}%)"
    )
    markdown_lines.append("")

    # By phase breakdown
    phase_summary = (
        summary_df.groupby("phase")
        .agg(
            {
                "input_tokens": "sum",
                "output_tokens": "sum",
                "total_tokens": "sum",
                "actual_cost_usd": "sum",
                "cost_fetch_success": "sum",
                "time_s": "sum",
            }
        )
        .round(6)
    )

    markdown_lines.append("## 📈 BY PHASE")
    markdown_lines.append("")
    for phase, row in phase_summary.iterrows():
        markdown_lines.append(f"### {phase}")
        markdown_lines.append("")
        markdown_lines.append(
            f"- **Tokens:** {row['input_tokens']:,} input + {row['output_tokens']:,} output = {row['total_tokens']:,} total"
        )
        markdown_lines.append(f"- **Actual cost:** ${row['actual_cost_usd']:.6f}")
        markdown_lines.append(f"- **Successful cost fetches:** {row['cost_fetch_success']:.0f}")
        markdown_lines.append("")

    # By model breakdown
    model_summary = (
        summary_df.groupby(["model_name"])
        .agg(
            {
                "input_tokens": "sum",
                "output_tokens": "sum",
                "total_tokens": "sum",
                "actual_cost_usd": "sum",
                "cost_fetch_success": "sum",
                "time_s": "sum",
            }
        )
        .round(6)
        .sort_values("total_tokens", ascending=False)
    )

    markdown_lines.append("## 🤖 BY MODEL (ranked by total tokens)")
    markdown_lines.append("")
    for model, row in model_summary.iterrows():
        markdown_lines.append(f"### {model}")
        markdown_lines.append("")
        markdown_lines.append(
            f"- **Tokens:** {row['input_tokens']:,} input + {row['output_tokens']:,} output = {row['total_tokens']:,} total"
        )
        markdown_lines.append(f"- **Actual cost:** ${row['actual_cost_usd']:.6f}")
        markdown_lines.append(f"- **Successful cost fetches:** {row['cost_fetch_success']:.0f}")
        markdown_lines.append("")

    # Cost efficiency analysis
    markdown_lines.append("## 💰 COST EFFICIENCY")
    markdown_lines.append("")
    if total_tokens > 0:
        cost_per_1k_tokens = (total_cost / total_tokens) * 1000
        markdown_lines.append(f"- **Average cost per 1K tokens:** ${cost_per_1k_tokens:.6f}")

    if len(model_summary) > 1:
        most_expensive = model_summary["actual_cost_usd"].idxmax()
        least_expensive = model_summary["actual_cost_usd"].idxmin()
        markdown_lines.append(
            f"- **Most expensive model:** {most_expensive} (${model_summary.loc[most_expensive, 'actual_cost_usd']:.6f})"
        )
        markdown_lines.append(
            f"- **Least expensive model:** {least_expensive} (${model_summary.loc[least_expensive, 'actual_cost_usd']:.6f})"
        )

    markdown_lines.append("")
    markdown_lines.append("## 📋 Notes")
    markdown_lines.append("")
    markdown_lines.append("- Using actual costs from OpenRouter's generation endpoint")
    markdown_lines.append(
        f"- Successful cost fetches: {successful_fetches}/{total_calls} calls ({successful_fetches/total_calls*100:.1f}%)"
    )
    if successful_fetches < total_calls:
        failed_calls = total_calls - successful_fetches
        markdown_lines.append(f"- {failed_calls} calls used fallback estimates")

    return "\n".join(markdown_lines)

In [ ]:
def fetch_actual_cost(generation_id: str, client: Any) -> dict[str, Any]:
    """
    Fetch actual cost and token information from OpenRouter's generation endpoint.
    
    Args:
        generation_id: The unique generation ID from the completion response
        client: OpenRouter API client (OpenAI-compatible)
        
    Returns:
        Dictionary containing actual cost and token data, or fallback values if error
    """
    import time
    
    # Wait a moment for the generation to be processed by OpenRouter
    time.sleep(0.5)
    
    try:
        # Extract API key and base URL from client
        api_key = client.api_key
        base_url = str(client.base_url).rstrip('/') if hasattr(client, 'base_url') else "https://openrouter.ai/api/v1"
        
        # Make direct request to generation endpoint
        import requests
        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
        
        response = requests.get(
            f"{base_url}/generation?id={generation_id}",
            headers=headers,
            timeout=10
        )
        
        if response.status_code == 200:
            data = response.json()
            gen_data = data.get('data', {})
            
            return {
                "generation_id": generation_id,
                "actual_cost_usd": gen_data.get('total_cost', 0.0),
                "actual_input_tokens": gen_data.get('native_tokens_prompt', 0),
                "actual_output_tokens": gen_data.get('native_tokens_completion', 0),
                "actual_total_tokens": (gen_data.get('native_tokens_prompt', 0) +
                                     gen_data.get('native_tokens_completion', 0)),
                "provider_name": gen_data.get('provider_name'),
                "model": gen_data.get('model'),
                "latency": gen_data.get('latency'),
                "generation_time": gen_data.get('generation_time'),
                "cache_discount": gen_data.get('cache_discount', 0.0),
                "finish_reason": gen_data.get('finish_reason'),
                "fetch_success": True
            }
        else:
            print(f"⚠️ Failed to fetch generation data for {generation_id}: HTTP {response.status_code}")
            return {
                "generation_id": generation_id,
                "actual_cost_usd": 0.0,
                "actual_input_tokens": 0,
                "actual_output_tokens": 0,
                "actual_total_tokens": 0,
                "provider_name": None,
                "model": None,
                "latency": None,
                "generation_time": None,
                "cache_discount": 0.0,
                "finish_reason": None,
                "fetch_success": False,
                "error": f"HTTP {response.status_code}"
            }
            
    except Exception as e:
        print(f"⚠️ Error fetching generation data for {generation_id}: {e}")
        return {
            "generation_id": generation_id,
            "actual_cost_usd": 0.0,
            "actual_input_tokens": 0,
            "actual_output_tokens": 0,
            "actual_total_tokens": 0,
            "provider_name": None,
            "model": None,
            "latency": None,
            "generation_time": None,
            "cache_discount": 0.0,
            "finish_reason": None,
            "fetch_success": False,
            "error": str(e)
        }

def fetch_all_actual_costs(df: pd.DataFrame, client: Any, generation_id_col: str = "generation_id") -> pd.DataFrame:
    """
    Fetch actual costs for all generation IDs in a DataFrame.
    
    Args:
        df: DataFrame containing generation_id column
        client: OpenRouter API client
        generation_id_col: Name of the column containing generation IDs
        
    Returns:
        Updated DataFrame with actual cost columns populated
    """
    import time
    
    print(f"\n🔍 Fetching actual costs for {len(df)} generations...")
    
    # Get unique generation IDs that are not None
    generation_ids = df[generation_id_col].dropna().unique()
    
    if len(generation_ids) == 0:
        print("⚠️ No generation IDs found to fetch costs for")
        return df
    
    print(f"  Found {len(generation_ids)} unique generation IDs to fetch")
    
    # Add a small delay to allow OpenRouter to process all generations
    print("  Waiting 2 seconds for OpenRouter to process generations...")
    time.sleep(2)
    
    # Fetch costs for each unique generation ID
    cost_data_map = {}
    for i, gen_id in enumerate(generation_ids, 1):
        print(f"  [{i}/{len(generation_ids)}] Fetching cost for {gen_id}...")
        cost_data = fetch_actual_cost(gen_id, client)
        cost_data_map[gen_id] = cost_data
    
    # Update DataFrame with fetched cost data
    for col in ["actual_cost_usd", "actual_input_tokens", "actual_output_tokens",
                "actual_total_tokens", "provider_name", "latency", "generation_time",
                "cache_discount", "finish_reason", "cost_fetch_success"]:
        if col not in df.columns:
            df[col] = None
    
    # Map the fetched data back to the DataFrame
    for idx, row in df.iterrows():
        gen_id = row[generation_id_col]
        if pd.notna(gen_id) and gen_id in cost_data_map:
            cost_data = cost_data_map[gen_id]
            df.at[idx, "actual_cost_usd"] = cost_data["actual_cost_usd"]
            df.at[idx, "actual_input_tokens"] = cost_data["actual_input_tokens"]
            df.at[idx, "actual_output_tokens"] = cost_data["actual_output_tokens"]
            df.at[idx, "actual_total_tokens"] = cost_data["actual_total_tokens"]
            df.at[idx, "provider_name"] = cost_data["provider_name"]
            df.at[idx, "latency"] = cost_data["latency"]
            df.at[idx, "generation_time"] = cost_data["generation_time"]
            df.at[idx, "cache_discount"] = cost_data["cache_discount"]
            df.at[idx, "finish_reason"] = cost_data["finish_reason"]
            df.at[idx, "cost_fetch_success"] = cost_data["fetch_success"]
    
    # Report success stats
    successful_fetches = df["cost_fetch_success"].sum() if "cost_fetch_success" in df.columns else 0
    total_cost = df["actual_cost_usd"].sum() if "actual_cost_usd" in df.columns else 0
    
    print("\n✓ Cost fetch complete:")
    print(f"  Successfully fetched: {successful_fetches}/{len(df)} rows")
    print(f"  Total actual cost: ${total_cost:.6f}")
    
    return df

def generate_cost_summary(
    questions_df: pd.DataFrame, answers_df: pd.DataFrame, ratings_df: pd.DataFrame
) -> pd.DataFrame:
    """
    Generate a comprehensive cost and token usage summary across all API calls.

    Args:
        questions_df: DataFrame with question generation data
        answers_df: DataFrame with answer generation data
        ratings_df: DataFrame with rating/evaluation data

    Returns:
        DataFrame with detailed cost and token breakdowns by phase and model
    """
    summary_data = []

    def process_df(df, phase, model_id_col, model_name_col, time_col):
        if "actual_total_tokens" in df.columns:
            for _, row in df.iterrows():
                if row["actual_total_tokens"] > 0:  # Skip error rows
                    
                    latency = round(row.get("latency") / 1000, 2) if row.get("latency") is not None else None
                    generation_time = round(row.get("generation_time") / 1000, 2) if row.get("generation_time") is not None else None
                    
                    processing_time = None
                    if latency is not None and generation_time is not None:
                        processing_time = round((latency + generation_time), 2)
                        
                    summary_data.append(
                        {
                            "phase": phase,
                            "model_id": row[model_id_col],
                            "model_name": row[model_name_col],
                            "input_tokens": row["actual_input_tokens"],
                            "output_tokens": row["actual_output_tokens"],
                            "total_tokens": row["actual_total_tokens"],
                            "actual_cost_usd": row["actual_cost_usd"],
                            "cost_fetch_success": row["cost_fetch_success"],
                            "generation_id": row.get("generation_id"),
                            "time_s": row[time_col],
                            "provider_name": row.get("provider_name"),
                            "latency": latency,
                            "generation_time": generation_time,
                            "processing_time": processing_time,
                            "cache_discount": row.get("cache_discount"),
                            "finish_reason": row.get("finish_reason"),
                        }
                    )

    process_df(questions_df, "Question Generation", "question_model_id", "question_model_name", "gen_time_s")
    process_df(answers_df, "Answer Generation", "answer_model_id", "answer_model_name", "answer_time_s")
    process_df(ratings_df, "Answer Rating", "evaluator_model_id", "evaluator_model_name", "evaluation_time_s")

    if not summary_data:
        print("⚠️ No token data found in any DataFrames")
        return pd.DataFrame()

    summary_df = pd.DataFrame(summary_data)

    return summary_df

In [ ]:
def generate_questions(models: pd.DataFrame, client: Any) -> pd.DataFrame:
    """
    Generate evaluation questions using the provided models.

    Each model generates one challenging question designed to test reasoning depth.

    Args:
        models: DataFrame with model information (must have 'model_id' and 'model_name' columns)
        client: OpenRouter API client (OpenAI-compatible)

    Returns:
        DataFrame with generated questions and metadata

    Example
        >>> all_q = generate_questions(top_models, client)
        >>> print(f"Generated {len(all_q)} valid questions out of {len(models)} attempts")
        Generated 5 valid questions out of 5 attempts

    Notes:
        - Uses temperature=0.8 for creative/diverse questions
        - Each question is limited to 10000 tokens
        - Errors are captured in the DataFrame but don't stop execution
        - Preview of each question is printed during generation
        - Stores generation IDs for later cost fetching
        - Call fetch_all_actual_costs() after generation to get actual cost data
    """
    question_generation_prompt = (
        "Please craft ONE challenging, original, nuanced question that can effectively "
        "discriminate between language models of varying reasoning depth. The question should: "
        "(1) require multi-step reasoning, (2) avoid simple trivia, (3) be answerable without external "
        "browsing, (4) not be purely opinion, (5) allow partial credit, and (6) be less than 400 tokens. Provide only the question text."
    )

    generated_questions: list[dict[str, Any]] = []

    # Each model generates one question
    for _, model in models.iterrows():
        mid = model["model_id"]
        mname = model.get("model_name", mid)
        print(f"\n[Generation] {mname} generating a question...")

        try:
            start = time.time()
            completion = client.chat.completions.create(
                model=mid,
                messages=[{"role": "user", "content": question_generation_prompt}],
                max_tokens=10000,  # Limit response length (increased to 10,000 for Gemini 2.5 Pro)
                temperature=0.8,  # Higher temperature for creative/diverse questions
            )
            q_text = completion.choices[0].message.content.strip()
            elapsed = time.time() - start

            # Extract basic token usage from completion response
            usage = completion.usage
            input_tokens = usage.prompt_tokens if usage else 0
            output_tokens = usage.completion_tokens if usage else 0
            total_tokens = usage.total_tokens if usage else 0

            # Get generation ID for later cost fetching
            generation_id = completion.id if hasattr(completion, 'id') else None
            
            generated_questions.append(
                {
                    "question_model_id": mid,
                    "question_model_name": mname,
                    "question": q_text,
                    "gen_time_s": round(elapsed, 2),
                    "generation_id": generation_id,
                    # Token counts from completion response (will be verified against actual later)
                    "input_tokens": input_tokens,
                    "output_tokens": output_tokens,
                    "total_tokens": total_tokens,
                }
            )
            
            print(f"✓ Question from {mname}: {q_text[:110]}{'...' if len(q_text) > 110 else ''}")
            print(f"  📊 Reported tokens: {input_tokens} input + {output_tokens} output = {total_tokens} total")
            print(f"  🔑 Generation ID: {generation_id}")
            
        except Exception as err:  # noqa: BLE001
            print(f"✗ {mname} failed: {err}")

    questions_df = pd.DataFrame(generated_questions)

    print(f"\n✓ Generated {len(questions_df)} valid questions out of {len(models)} attempts")
    print("  💡 Use fetch_all_actual_costs(questions_df, client) to retrieve actual cost data")

    return questions_df

### 💰 Cost Tracking Pattern

**New workflow for fetching actual costs:**

1. **Generate content** (questions/answers/evaluations) - stores `generation_id` in DataFrame
2. **Wait briefly** - allows OpenRouter to process all generations  
3. **Fetch costs in batch** - use `fetch_all_actual_costs(df, client)` to retrieve actual cost data

**Benefits:**
- ✅ Avoids immediate API calls that may return incomplete data
- ✅ Batch processing is more efficient 
- ✅ Separates content generation from cost tracking
- ✅ Can re-run cost fetching if needed without regenerating content

**Example:**
```python
# Step 1: Generate questions (stores generation_id)
questions_df = generate_questions(top_models, client)

# Step 2: Fetch actual costs (updates DataFrame with cost data)
questions_df_full = fetch_all_actual_costs(questions_df, client)

# Now questions_df_full has columns: actual_cost_usd, actual_input_tokens, 
# actual_output_tokens, provider_name, latency, etc.
```

In [ ]:
questions_df = generate_questions(top_models, client)

In [ ]:
# Fetch actual costs for all generated questions
# OpenRouter has a slight delay before generation data is available
questions_df_full = fetch_all_actual_costs(questions_df, client)

In [ ]:
questions_df_full

In [ ]:
def answer_questions(questions_df: pd.DataFrame, models: pd.DataFrame, client: Any) -> pd.DataFrame:
    """
    Generate answers to questions using the provided models.

    Each model answers every question from every model (including its own question).

    Args:
        questions_df: DataFrame with questions (must have 'question' and 'question_model_name' columns)
        models: DataFrame with model information (must have 'model_id' and 'model_name' columns)
        client: OpenRouter API client (OpenAI-compatible)

    Returns:
        DataFrame with answers containing columns: question_model_name, question,
        answer_model_id, answer_model_name, answer, answer_time_s, generation_id, etc.
        
    Notes:
        - Stores generation IDs for later cost fetching
        - Call fetch_all_actual_costs() after generation to get actual cost data
    """
    answers: list[dict[str, Any]] = []
    answer_instructions = (
        "You will be given a question designed to evaluate reasoning depth. Provide a thorough, "
        "structured answer. Show reasoning explicitly if helpful, but keep it concise and logical."
        "If you have to iterate more than 5 times, admit you are stuck and provide your best possible answer."
    )

    # Each model answers each question
    for _, qrow in questions_df.iterrows():
        q_text = qrow["question"]
        origin_model = qrow["question_model_name"]

        for _, model in models.iterrows():
            mid = model["model_id"]
            mname = model.get("model_name", mid)
            print(f"\n[Answer] {mname} answering question from {origin_model}...")

            try:
                start = time.time()
                completion = client.chat.completions.create(
                    model=mid,
                    messages=[
                        {"role": "system", "content": answer_instructions},
                        {"role": "user", "content": q_text},
                    ],
                    max_tokens=20000,  # Allow longer responses for thorough answers (especially for Gemini 2.5 Pro)
                    temperature=0.5,  # Moderate temperature for balanced responses
                    timeout=30.0,  # 30 second timeout to prevent hanging
                )
                ans_text = completion.choices[0].message.content.strip()
                elapsed = time.time() - start

                # Extract basic token usage from completion response
                usage = completion.usage
                input_tokens = usage.prompt_tokens if usage else 0
                output_tokens = usage.completion_tokens if usage else 0
                total_tokens = usage.total_tokens if usage else 0
                
                # Get generation ID for later cost fetching
                generation_id = completion.id if hasattr(completion, 'id') else None
                
                answers.append(
                    {
                        "question_model_name": origin_model,
                        "question": q_text,
                        "answer_model_id": mid,
                        "answer_model_name": mname,
                        "answer": ans_text,
                        "answer_time_s": round(elapsed, 2),
                        "generation_id": generation_id,
                        # Token counts from completion response (will be verified against actual later)
                        "input_tokens": input_tokens,
                        "output_tokens": output_tokens,
                        "total_tokens": total_tokens,
                    }
                )
                
                print(f"✓ Answer length: {len(ans_text)} chars")
                print(f"  📊 Reported tokens: {input_tokens} input + {output_tokens} output = {total_tokens} total")
                print(f"  🔑 Generation ID: {generation_id}")
                    
            except Exception as err:  # noqa: BLE001
                # Record errors but continue with other models
                answers.append(
                    {
                        "question_model_name": origin_model,
                        "question": q_text,
                        "answer_model_id": mid,
                        "answer_model_name": mname,
                        "answer": f"Error: {err}",
                        "answer_time_s": None,
                        "generation_id": None,
                        "input_tokens": 0,
                        "output_tokens": 0,
                        "total_tokens": 0,
                    }
                )
                print(f"✗ {mname} failed to answer: {err}")

    answers_df = pd.DataFrame(answers)
    
    print(f"\n✓ Answers DataFrame ready with {len(answers_df)} answers")
    print("  💡 Use fetch_all_actual_costs(answers_df, client) to retrieve actual cost data")

    return answers_df

In [ ]:
answers_df = answer_questions(questions_df_full, top_models, client)

In [ ]:
# Fetch actual costs for all generated answers
answers_df_full = fetch_all_actual_costs(answers_df, client)

In [ ]:
answers_df_full

In [ ]:
# spot check various answers, just change the index number (0 based )
print(answers_df_full.iloc[23]["answer"])

In [ ]:
def evaluate_answers(answers_df: pd.DataFrame, models: pd.DataFrame, client: Any) -> pd.DataFrame:
    """
    Generate ratings for answers using the evaluation models.

    Each model rates every answer given by every model (including its own).

    Args:
        answers_df: DataFrame with answers (must have columns for question, answer, etc.)
        models: DataFrame with model information (must have 'model_id' and 'model_name' columns)
        client: OpenRouter API client (OpenAI-compatible)

    Returns:
        DataFrame with evaluations containing columns: question_model_name, question,
        answer_model_name, answer, evaluator_model_id, evaluator_model_name, rating,
        explanation, evaluation_time_s, generation_id, etc.
        
    Notes:
        - Stores generation IDs for later cost fetching
        - Call fetch_all_actual_costs() after generation to get actual cost data
    """
    evaluations: list[dict[str, Any]] = []
    # rating_instructions = (
    #     "You are evaluating the quality of an answer to a reasoning-focused question. "
    #     "Rate the answer on a scale from 1-5 (integers only, 5 = outstanding).\n"
    #     "1 = Poor (incorrect, unhelpful, or off-topic)\n"
    #     "2 = Below average (partially correct but significant issues)\n"
    #     "3 = Average (mostly correct but lacking depth or clarity)\n"
    #     "4 = Good (correct, clear, and helpful)\n"
    #     "5 = Excellent (exceptional quality, comprehensive, insightful)\n\n"
    #     "Provide ONLY the numeric rating (1-5) followed by a brief explanation. "
    #     "Format: Rating: X\nExplanation: [your reasoning]"
    # )

    rating_instructions = (
        "You are evaluating the quality of an answer to a reasoning-focused question. "
        "Rate the answer on a scale from 1-10 (integers only, 10 = outstanding).\n"
        "Criteria (roughly equal weight):\n"
        "1. Clarity & organization\n"
        "2. Depth & correctness\n"
        "3. Completeness\n"
        "4. Insight/originality (if applicable)\n\n"
        "Provide ONLY the numeric rating (1-10) followed by a brief explanation.\n"
        "Format: Rating: X\nExplanation: [your reasoning]"
    )

    # Each model evaluates each answer
    for _, ans_row in answers_df.iterrows():
        question = ans_row["question"]
        answer = ans_row["answer"]
        q_model = ans_row["question_model_name"]
        a_model = ans_row["answer_model_name"]

        for _, model in models.iterrows():
            mid = model["model_id"]
            mname = model.get("model_name", mid)
            print(f"\n[Evaluate] {mname} rating answer from {a_model} to {q_model}'s question...")

            try:
                eval_prompt = f"Question: {question}\n\nAnswer: {answer}"
                
                start = time.time()
                completion = client.chat.completions.create(
                    model=mid,
                    messages=[
                        {"role": "system", "content": rating_instructions},
                        {"role": "user", "content": eval_prompt},
                    ],
                    max_tokens=10000,  # Shorter responses for evaluations
                    temperature=0.1,  # Low temperature for consistent evaluations
                    timeout=30.0,  # 30 second timeout to prevent hanging
                )
                eval_text = completion.choices[0].message.content.strip()
                elapsed = time.time() - start

                # Extract rating from the response
                rating = None
                explanation = eval_text
                try:
                    if "Rating:" in eval_text:
                        parts = eval_text.split("Rating:", 1)[1].split("\n", 1)
                        rating_str = parts[0].strip()
                        rating = int(rating_str)
                        if len(parts) > 1 and "Explanation:" in parts[1]:
                            explanation = parts[1].split("Explanation:", 1)[1].strip()
                    else:
                        # Try to extract number from beginning
                        import re
                        match = re.search(r'\b([1-5])\b', eval_text)
                        if match:
                            rating = int(match.group(1))
                except (ValueError, IndexError):
                    rating = None  # Will be handled as parsing error

                # Extract basic token usage from completion response
                usage = completion.usage
                input_tokens = usage.prompt_tokens if usage else 0
                output_tokens = usage.completion_tokens if usage else 0
                total_tokens = usage.total_tokens if usage else 0
                
                # Get generation ID for later cost fetching
                generation_id = completion.id if hasattr(completion, 'id') else None
                
                evaluations.append(
                    {
                        "question_model_name": q_model,
                        "question": question,
                        "answer_model_id": ans_row["answer_model_id"],
                        "answer_model_name": a_model,
                        "answer": answer,
                        "evaluator_model_id": mid,
                        "evaluator_model_name": mname,
                        "rating": rating,
                        "explanation": explanation,
                        "full_evaluation": eval_text,
                        "evaluation_time_s": round(elapsed, 2),
                        "generation_id": generation_id,
                        # Token counts from completion response (will be verified against actual later)
                        "input_tokens": input_tokens,
                        "output_tokens": output_tokens,
                        "total_tokens": total_tokens,
                    }
                )
                
                print(f"✓ Rating: {rating} | Explanation length: {len(explanation)} chars")
                print(f"  📊 Reported tokens: {input_tokens} input + {output_tokens} output = {total_tokens} total")
                print(f"  🔑 Generation ID: {generation_id}")
                    
            except Exception as err:  # noqa: BLE001
                # Record errors but continue with other models
                evaluations.append(
                    {
                        "question_model_name": q_model,
                        "question": question,
                        "answer_model_id": ans_row["answer_model_id"],
                        "answer_model_name": a_model,
                        "answer": answer,
                        "evaluator_model_id": mid,
                        "evaluator_model_name": mname,
                        "rating": None,
                        "explanation": f"Error: {err}",
                        "full_evaluation": f"Error: {err}",
                        "evaluation_time_s": None,
                        "generation_id": None,
                        "input_tokens": 0,
                        "output_tokens": 0,
                        "total_tokens": 0,
                    }
                )
                print(f"✗ {mname} failed to evaluate: {err}")

    ratings_df = pd.DataFrame(evaluations)
    
    print(f"\n✓ Ratings DataFrame ready with {len(ratings_df)} evaluations")
    print("  💡 Use fetch_all_actual_costs(ratings_df, client) to retrieve actual cost data")

    return ratings_df

### Understanding the Rating System

Each model evaluates answers on a **10-point scale** across these criteria:

1. **Clarity & Organization** (2.5 points)
   - Is the answer well-structured and easy to follow?
   - Are concepts explained clearly without unnecessary jargon?

2. **Depth & Correctness** (2.5 points)
   - Is the reasoning sound and logically valid?
   - Are facts accurate and relevant?

3. **Completeness** (2.5 points)
   - Does the answer address all parts of the question?
   - Are edge cases or caveats mentioned when appropriate?

4. **Insight & Originality** (2.5 points)
   - Does the answer provide novel perspectives or connections?
   - Is there evidence of deeper understanding beyond surface-level knowledge?

**Why use models as raters?**
- Consistent evaluation criteria across all answers
- Faster than human evaluation for large-scale comparisons
- Tests if models can accurately judge reasoning quality (meta-evaluation)

**Limitations:**
- Models may have biases (e.g., preferring similar styles to their own)
- Some subjective criteria may be interpreted differently
- This is why we aggregate ratings across multiple model-raters

In [ ]:
ratings_df = evaluate_answers(answers_df_full, top_models, client)

In [ ]:
# Fetch actual costs for all generated ratings
ratings_df_full = fetch_all_actual_costs(ratings_df, client)

In [ ]:
ratings_df_full

In [ ]:
cost_summary_df = generate_cost_summary(questions_df_full, answers_df_full, ratings_df_full)

In [ ]:
from IPython.display import display, Markdown

display(Markdown(generate_cost_summary_markdown(cost_summary_df)))

In [ ]:
cost_summary_df

In [ ]:
def generate_summary(ratings_df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Generate aggregation summaries
    summary_model = pd.DataFrame()
    summary_pair = pd.DataFrame()

    if not ratings_df.empty:
        # Calculate average rating for each model's answers
        summary_model = (
            ratings_df.groupby("answer_model_name")["rating"]
            .mean()
            .reset_index()
            .rename(columns={"rating": "avg_rating"})
        )
        
        # Calculate average rating for each (answerer, evaluator) pair
        summary_pair = (
            ratings_df.groupby(["answer_model_name", "evaluator_model_name"])["rating"]
            .mean()
            .reset_index()
            .rename(columns={"rating": "avg_rating"})
        )
        
        print("\n" + "="*80)
        print("Average rating per answer model:")
        print("="*80)
        display(summary_model.sort_values("avg_rating", ascending=False))
        
        print("\n" + "="*80)
        print("Cross-model rating matrix (long form):")
        print("="*80)
        display(summary_pair.head(20))
    else:
        print("⚠ No ratings captured.")

    return summary_model, summary_pair

In [ ]:
summary_model, summary_pair = generate_summary(ratings_df_full)

In [ ]:
summary_model

In [ ]:
summary_pair

### Summary Functions that generate markdown

In [ ]:
# Convert pivot table to markdown format
def pivot_to_markdown(pivot_df):
    """Convert a pivot table DataFrame to markdown table format."""
    if pivot_df.empty:
        return "No data available"

    # Start with header row
    headers = ["Model"] + list(pivot_df.columns)
    markdown_lines = []

    # Create header
    header_line = "| " + " | ".join(headers) + " |"
    markdown_lines.append(header_line)

    # Create separator line
    separator = "| " + " | ".join(["---"] * len(headers)) + " |"
    markdown_lines.append(separator)

    # Add data rows
    for index, row in pivot_df.iterrows():
        row_data = [str(index)]
        for col in pivot_df.columns:
            value = row[col]
            if pd.isna(value):
                row_data.append("N/A")
            else:
                row_data.append(f"{value:.2f}")
        row_line = "| " + " | ".join(row_data) + " |"
        markdown_lines.append(row_line)

    return "\n".join(markdown_lines)

def top_models_markdown(top_models: pd.DataFrame) -> str:
    """Generate a markdown table of top models with selected columns."""
    if top_models.empty:
        return "⚠️ No model data available"
    
    markdown_lines = []
    markdown_lines.append("## 🏆 Top Models Selected for Evaluation\n")
    markdown_lines.append("| Rank | Model Name | Tokens Used | Token Change |")
    markdown_lines.append("|------|------------|-------------|--------------|")
    
    for _, row in top_models.iterrows():
        rank = row['rank']
        model_name = row['model_name']
        tokens = row['tokens_formatted']
        change = row['token_change']
        markdown_lines.append(f"| {rank} | {model_name} | {tokens} | {change} |")
    
    return "\n".join(markdown_lines)

def questions_to_markdown(questions_df):
    """Convert generated questions DataFrame to markdown format."""
    if questions_df.empty:
        return "No questions available"

    markdown_lines = []
    markdown_lines.append("## 📝 Generated Evaluation Questions")
    markdown_lines.append("")

    for idx, row in questions_df.iterrows():
        markdown_lines.append(f"### {idx + 1}. Question by {row['question_model_name']}")
        markdown_lines.append(f"**Generation Time:** {row['gen_time_s']}s")
        if "actual_total_tokens" in row and row["actual_total_tokens"] > 0:
            markdown_lines.append(f"**Tokens Used:** {row['actual_input_tokens']} in + {row['actual_output_tokens']} out = {row['actual_total_tokens']} total")
            markdown_lines.append(f"**Actual Cost:** ${row['actual_cost_usd']:.4f}")
        markdown_lines.append(f"{row['question']}")
        markdown_lines.append("\n")

    return "\n".join(markdown_lines)

def model_performance_summary_markdown(summary_model: pd.DataFrame) -> str:
    """Generate a model performance summary in markdown format."""
    if summary_model.empty:
        return "⚠️ No model performance data available"

    markdown_lines = []
    markdown_lines.append("## 📊 Model Performance Summary\n")
    markdown_lines.append("### Overall Average Ratings (10-point scale)\n")

    for idx, row in summary_model.sort_values("avg_rating", ascending=False).iterrows():
        markdown_lines.append(f"{idx + 1}. **{row['answer_model_name']}**: {row['avg_rating']:.2f}/10")

    # markdown_lines.append("### Model Stats)\n")
    # for idx, row in summary_model.iterrows():
    #     markdown_lines.append(f"### {idx + 1}. {row['answer_model_name']}")
    #     markdown_lines.append(f"- **Average Response Time:** {row['avg_response_time_s']}s")
    #     markdown_lines.append(f"- **Total Tokens Used:** {row['total_tokens']}")
    #     markdown_lines.append(f"- **Total Cost:** ${row['total_cost_usd']:.4f}")
    #     markdown_lines.append("")

    return "\n".join(markdown_lines)

def cost_summary_markdown(cost_summary_df: pd.DataFrame) -> str:
    """Generate a cost summary in markdown format."""
    if cost_summary_df.empty:
        return "⚠️ No cost data available"

    markdown_lines = []
    markdown_lines.append("## 💲 Cost Summary and Token Usage\n")
    
    total_cost = cost_summary_df["actual_cost_usd"].sum()
    total_tokens = cost_summary_df["total_tokens"].sum()
    input_tokens = cost_summary_df["input_tokens"].sum()
    output_tokens = cost_summary_df["output_tokens"].sum()

    markdown_lines.append(f"- **Total Cost Across All Phases:** ${total_cost:.4f}")
    markdown_lines.append(f"- **Total Tokens Used Across All Phases:** {total_tokens:,}")
    markdown_lines.append(f"- **Total Input Tokens:** {input_tokens:,}")
    markdown_lines.append(f"- **Total Output Tokens:** {output_tokens:,}")
    markdown_lines.append("")

    return "\n".join(markdown_lines)

def generate_performance_tables_markdown(cost_summary_df: pd.DataFrame) -> str:
    """
    Generate performance metrics tables showing average cost, time, latency, and tokens
    by model and phase in markdown format.
    
    Args:
        cost_summary_df: DataFrame with columns: phase, model_name, actual_cost_usd, 
                        time_s, latency, generation_time, total_tokens, processing_time
    
    Returns:
        Markdown string with four performance metric tables
    """
    if cost_summary_df.empty:
        return "⚠️ No cost summary data available"
    
    markdown_lines = []
    markdown_lines.append("## 📊 Performance Metrics by Model and Phase\n")
    
    # Define the metrics to generate tables for
    metrics = [
        {
            "title": "Average Cost ($)",
            "column": "actual_cost_usd",
            "format": lambda x: f"{x:.4f}"
        },
        {
            "title": "Average Time (s)",
            "column": "time_s",
            "format": lambda x: f"{x:.2f}"
        },
        {
            "title": "Average Latency + Gen (s)",
            "column": "processing_time",
            "format": lambda x: f"{x:.2f}" if pd.notna(x) else "N/A"
        },
        {
            "title": "Average Tokens",
            "column": "total_tokens",
            "format": lambda x: f"{int(x)}"
        }
    ]
    
    # Generate a table for each metric
    for metric in metrics:
        markdown_lines.append(f"### {metric['title']}\n")
        
        # Pivot the data: rows = models, columns = phases
        pivot_df = cost_summary_df.pivot_table(
            index='model_name',
            columns='phase',
            values=metric['column'],
            aggfunc='mean'
        )
        
        # Reorder columns to match expected order
        phase_order = ['Question Generation', 'Answer Generation', 'Answer Rating']
        pivot_df = pivot_df[[col for col in phase_order if col in pivot_df.columns]]
        
        # Calculate overall average across all phases
        pivot_df['Overall Average'] = pivot_df.mean(axis=1)
        
        # Generate markdown table header
        headers = ["Model"] + list(pivot_df.columns)
        markdown_lines.append("| " + " | ".join(headers) + " |")
        markdown_lines.append("| " + " | ".join(["---"] * len(headers)) + " |")
        
        # Generate table rows
        for model_name, row in pivot_df.iterrows():
            row_values = [str(model_name)]
            for col in pivot_df.columns:
                value = row[col]
                if pd.notna(value):
                    row_values.append(metric['format'](value))
                else:
                    row_values.append("N/A")
            markdown_lines.append("| " + " | ".join(row_values) + " |")
        
        markdown_lines.append("")  # Empty line between tables
    
    return "\n".join(markdown_lines)

def cross_model_rating_matrix_markdown(summary_pair: pd.DataFrame) -> str:
    """Generate a cross-model rating matrix in markdown format."""
    if summary_pair.empty:
        return "⚠️ No cross-model rating data available"

    # Pivot the DataFrame to create a matrix
    pivot_df = summary_pair.pivot(index="answer_model_name", columns="evaluator_model_name", values="avg_rating")

    markdown_lines = []
    markdown_lines.append("## 🤝 Cross-Model Rating Matrix\n")
    markdown_lines.append("This shows how each model (rater/columns) rated each model's answers (answerer/rows):\n")
    markdown_lines.append(pivot_to_markdown(pivot_df))
    markdown_lines.append("")

    return "\n".join(markdown_lines)

def cost_breakdown_by_phase_markdown(cost_summary_df: pd.DataFrame) -> str:
    """Generate a cost breakdown by evaluation phase in markdown format."""
    if cost_summary_df.empty:
        return "⚠️ Run the full pipeline first to see cost breakdown\n   Execute the cell: 'Run the complete evaluation pipeline with cost tracking'"

    markdown_lines = []
    markdown_lines.append("## 💰 Cost Breakdown by Phase")
    markdown_lines.append("")

    for phase in ["Question Generation", "Answer Generation", "Answer Rating"]:
        phase_data = cost_summary_df[cost_summary_df["phase"] == phase]
        if not phase_data.empty:
            total_calls = len(phase_data)
            total_tokens = phase_data["total_tokens"].sum()
            total_cost = phase_data["actual_cost_usd"].sum()
            avg_tokens_per_call = total_tokens / total_calls if total_calls > 0 else 0
            avg_cost_per_call = total_cost / total_calls if total_calls > 0 else 0

            markdown_lines.append(f"### 📊 {phase}\n")
            markdown_lines.append(f"- **API calls**: {total_calls}")
            markdown_lines.append(f"- **Total tokens**: {total_tokens:,}")
            markdown_lines.append(f"- **Total cost**: ${total_cost:.4f}")
            markdown_lines.append(f"- **Avg tokens/call**: {avg_tokens_per_call:.0f}")
            markdown_lines.append(f"- **Avg cost/call**: ${avg_cost_per_call:.4f}")
            markdown_lines.append("")

    return "\n".join(markdown_lines)

def most_expensive_calls_markdown(cost_summary_df: pd.DataFrame) -> str:
    """Generate a summary of the most expensive individual API calls in markdown format."""
    if cost_summary_df.empty:
        return "⚠️ Run the full pipeline first to see expensive calls\n   Execute the cell: 'Run the complete evaluation pipeline with cost tracking'"

    markdown_lines = []
    markdown_lines.append("## 💸 Most Expensive API Calls")
    markdown_lines.append("")

    top_expensive = cost_summary_df.nlargest(5, "actual_cost_usd")
    for idx, row in top_expensive.iterrows():
        markdown_lines.append(f"### {idx + 1}. {row['model_name']} ({row['phase']})")
        markdown_lines.append("")
        markdown_lines.append(f"- **Tokens**: {row['input_tokens']:,} in + {row['output_tokens']:,} out = {row['total_tokens']:,}")
        markdown_lines.append(f"- **Cost**: ${row['actual_cost_usd']:.4f}")
        markdown_lines.append("")

    return "\n".join(markdown_lines)

def generate_rating_statistics_markdown(ratings_df: pd.DataFrame) -> str:
    """Generate rating statistics in markdown format."""
    if ratings_df.empty or "rating" not in ratings_df.columns:
        return "⚠️ **No valid ratings data available**"

    # Overall statistics
    valid_ratings = ratings_df["rating"].dropna()

    markdown_lines = []
    markdown_lines.append("## Rating Statistics Summary")
    markdown_lines.append("")

    markdown_lines.append("### 📊 Overall Rating Distribution")
    markdown_lines.append("")
    markdown_lines.append(f"- **Total ratings collected**: {len(valid_ratings)}")
    markdown_lines.append(f"- **Average rating**: {valid_ratings.mean():.2f} / 10")
    markdown_lines.append(f"- **Median rating**: {valid_ratings.median():.1f} / 10")
    markdown_lines.append(f"- **Standard deviation**: {valid_ratings.std():.2f}")
    markdown_lines.append(f"- **Range**: {valid_ratings.min():.0f} - {valid_ratings.max():.0f}")
    markdown_lines.append("")

    # Rating distribution
    markdown_lines.append("### 📈 Rating Frequency")
    markdown_lines.append("")
    rating_counts = valid_ratings.value_counts().sort_index(ascending=False)
    for score, count in rating_counts.items():
        bar = "█" * int(count / len(valid_ratings) * 20)  # Shorter bars for markdown
        percentage = count / len(valid_ratings) * 100
        markdown_lines.append(f"- **{score:2.0f}/10**: `{bar}` ({count} ratings, {percentage:.1f}%)")
    markdown_lines.append("")

    # Self-rating analysis (do models rate themselves higher?)
    if "answer_model_name" in ratings_df.columns and "rater_model_name" in ratings_df.columns:
        self_ratings = ratings_df[ratings_df["answer_model_name"] == ratings_df["rater_model_name"]][
            "rating"
        ].dropna()

        other_ratings = ratings_df[ratings_df["answer_model_name"] != ratings_df["rater_model_name"]][
            "rating"
        ].dropna()

        if len(self_ratings) > 0 and len(other_ratings) > 0:
            markdown_lines.append("### 🤔 Self-Rating vs. Peer-Rating")
            markdown_lines.append("")
            markdown_lines.append(f"- **Average when rating own answers**: {self_ratings.mean():.2f}")
            markdown_lines.append(f"- **Average when rating others' answers**: {other_ratings.mean():.2f}")
            diff = self_ratings.mean() - other_ratings.mean()

            if abs(diff) < 0.5:
                bias_text = f"Models are fairly unbiased (difference: {diff:+.2f})"
            elif diff > 0:
                bias_text = f"Models tend to rate themselves higher (difference: {diff:+.2f})"
            else:
                bias_text = f"Models tend to rate themselves lower (difference: {diff:+.2f})"

            markdown_lines.append(f"- **Bias Analysis**: {bias_text}")
            markdown_lines.append("")

    # Most generous and strictest raters
    if "rater_model_name" in ratings_df.columns:
        avg_by_rater = ratings_df.groupby("rater_model_name")["rating"].mean().sort_values(ascending=False)

        markdown_lines.append("### 🎭 Rater Characteristics")
        markdown_lines.append("")
        markdown_lines.append(
            f"- **🎁 Most Generous Rater**: {avg_by_rater.index[0]} (avg: {avg_by_rater.iloc[0]:.2f})"
        )
        markdown_lines.append(f"- **🔍 Strictest Rater**: {avg_by_rater.index[-1]} (avg: {avg_by_rater.iloc[-1]:.2f})")
        markdown_lines.append(f"- **📏 Rater Spread**: {avg_by_rater.iloc[0] - avg_by_rater.iloc[-1]:.2f} points")

    return "\n".join(markdown_lines)


In [ ]:
def get_highest_rated_answers(ratings_df: pd.DataFrame, cost_summary_df: pd.DataFrame) -> pd.DataFrame:
    """
    Find the highest rated answer for each question.
    In case of a tie, select the cheapest one based on actual_cost_usd.
    
    Args:
        ratings_df: DataFrame with ratings data
        cost_summary_df: DataFrame with cost data
        
    Returns:
        DataFrame with the highest rated answer for each question
    """
    # Merge ratings with cost data to get actual_cost_usd for each answer
    # We need to match on the answer generation phase
    answer_costs = cost_summary_df[cost_summary_df['phase'] == 'Answer Generation'].copy()
    
    # Merge on answer_model_name (from ratings_df) with model_name (from cost_summary_df)
    ratings_with_cost = ratings_df.merge(
        answer_costs[['model_name', 'actual_cost_usd']],
        left_on='answer_model_name',
        right_on='model_name',
        how='left',
        suffixes=('', '_answer_cost')
    )
    
    # For each question, get the average rating per answer model
    # Preserve answer, explanation, and full_evaluation columns using 'first'
    # For evaluator info, we'll take first evaluator and sum evaluation times
    avg_ratings = ratings_with_cost.groupby(['question_model_name', 'question', 'answer_model_name']).agg({
        'rating': 'mean',
        'actual_cost_usd': 'first',  # Take first value since it's the same for all ratings of same answer
        'answer': 'first',  # These are the same for all ratings of the same answer
        'explanation': 'first',
        'full_evaluation': 'first',
        'evaluator_model_name': 'first',
        'evaluation_time_s': 'first'
        # 'evaluator_model_name': lambda x: ', '.join(sorted(set(x))),  # List all unique evaluators
        # 'evaluation_time_s': 'sum'  # Sum of all evaluation times for this answer
    }).reset_index()
    
    # Sort by rating (descending) and cost (ascending) to handle ties
    avg_ratings['actual_cost_usd'] = pd.to_numeric(avg_ratings['actual_cost_usd'], errors='coerce')
    avg_ratings = avg_ratings.sort_values(
        ['question_model_name', 'rating', 'actual_cost_usd'],
        ascending=[True, False, True]
    )
    
    # Get the top answer for each question
    highest_rated = avg_ratings.groupby('question_model_name').first().reset_index()
    
    # Add rank to show position
    highest_rated['rank'] = range(1, len(highest_rated) + 1)
    
    return highest_rated[['rank', 'question_model_name', 'answer_model_name', 'rating', 'actual_cost_usd', 'question', 'answer', 'explanation', 'full_evaluation', 'evaluator_model_name', 'evaluation_time_s']]

In [ ]:
def generate_answer_evaluation_markdown(question_model_name: str, answer_model_name: str, ratings_df: pd.DataFrame) -> str:
    """
    Generate a markdown-formatted summary of answer evaluations.
    
    Shows the question and answer once at the top, followed by individual
    evaluations from each rater model. Selects evaluations for the specified
    question and answer models.
    
    Args:
        question_model_name: Name of the question model
        answer_model_name: Name of the answer model
        ratings_df: DataFrame with evaluation data (typically filtered to one question/answer pair)
    Returns:
        Markdown-formatted string
    """
    df = ratings_df[
        (ratings_df['question_model_name'] == question_model_name) &
        (ratings_df['answer_model_name'] == answer_model_name)
    ]

    if df.empty:
        return "⚠️ No evaluation data available"
    
    markdown_lines = []
    
    # Get common values (should be the same across all rows)
    first_row = df.iloc[0]
    question_model = first_row['question_model_name']
    question = first_row['question']
    answer_model = first_row['answer_model_name']
    answer = first_row['answer']
    
    # Header
    markdown_lines.append("## Answer Evaluation Summary")
    markdown_lines.append("")
    
    # Question section
    markdown_lines.append("## 📝 Question\n")
    markdown_lines.append(f"**Asked by:** {question_model}")
    markdown_lines.append("")
    markdown_lines.append(question)
    markdown_lines.append("")
    
    # Answer section
    markdown_lines.append("## 💡 Answer\n")
    markdown_lines.append(f"**Answered by:** {answer_model}")
    markdown_lines.append("")
    markdown_lines.append(answer)
    markdown_lines.append("")
    
    # Evaluations section
    markdown_lines.append("## 🎯 Evaluations")
    markdown_lines.append("")
    
    # Sort by rating (highest first) for easier reading
    sorted_df = df.sort_values('rating', ascending=False)
    
    for _, row in sorted_df.iterrows():
        evaluator = row['evaluator_model_name']
        rating = row['rating']
        explanation = row['explanation']
        full_eval = row['full_evaluation']
        eval_time = row['evaluation_time_s']
        
        markdown_lines.append(f"### {evaluator} - Rating: {rating}/10")
        markdown_lines.append("")
        markdown_lines.append(f"*Evaluation time: {eval_time}s*")
        markdown_lines.append("")
        markdown_lines.append("**Explanation:**")
        markdown_lines.append(explanation)
        markdown_lines.append("")
        
        # Optionally include full evaluation if different from explanation
        if full_eval != explanation and pd.notna(full_eval):
            markdown_lines.append("<details>")
            markdown_lines.append("<summary>Full Evaluation Text</summary>")
            markdown_lines.append("")
            markdown_lines.append(full_eval)
            markdown_lines.append("")
            markdown_lines.append("</details>")
            markdown_lines.append("")
    
    # Summary statistics
    markdown_lines.append("---")
    markdown_lines.append("")
    markdown_lines.append("## 📊 Summary Statistics")
    markdown_lines.append("")
    markdown_lines.append(f"- **Number of Evaluators:** {len(df)}")
    markdown_lines.append(f"- **Average Rating:** {df['rating'].mean():.2f}/10")
    markdown_lines.append(f"- **Rating Range:** {df['rating'].min()}-{df['rating'].max()}")
    markdown_lines.append(f"- **Total Evaluation Time:** {df['evaluation_time_s'].sum():.2f}s")
    
    return "\n".join(markdown_lines)

### Grab a sample question and answer from a couple of the top models to see question, answer, and evaluation analysis

In [ ]:
hard_question_markdown = generate_answer_evaluation_markdown("Grok 4 Fast", "Gemini 2.5 Flash", ratings_df_full)

display(Markdown(hard_question_markdown))

In [ ]:
# Get the highest rated answers
highest_rated_answers = get_highest_rated_answers(ratings_df_full, cost_summary_df)

print("🏆 Highest Rated Answer for Each Question")
print("=" * 80)
print("\nBest answers selected based on:")
print("  1. Highest average rating across all evaluators")
print("  2. Lowest cost (in case of rating ties)")
print()

display(highest_rated_answers)

In [ ]:
highest_rated_answers

In [ ]:
def generate_highest_rated_answers_report(ratings_df: pd.DataFrame, cost_summary_df: pd.DataFrame) -> str:
    """
    Generate a comprehensive markdown report for all highest-rated answers.
    
    Loops through the highest_rated_answers DataFrame, generates an evaluation
    summary for each question/answer pair, and aggregates them into a single report.
    
    Args:
        ratings_df: DataFrame with all ratings data
        cost_summary_df: DataFrame with cost data
        
    Returns:
        Aggregated markdown string containing all answer evaluations
    """
    highest_rated_answers = get_highest_rated_answers(ratings_df, cost_summary_df)

    if highest_rated_answers.empty:
        return "⚠️ No highest-rated answers available"
    
    markdown_sections = []
    
    # Add report header
    markdown_sections.append("## 🏆 Highest Rated Answers - Detailed Evaluations")
    markdown_sections.append("")
    markdown_sections.append(f"This report contains detailed evaluations for the {len(highest_rated_answers)} highest-rated answers.")
    markdown_sections.append("")
    markdown_sections.append("---")
    markdown_sections.append("")
    
    # Loop through each highest-rated answer
    for idx, row in highest_rated_answers.iterrows():
        question_model = row['question_model_name']
        answer_model = row['answer_model_name']
        avg_rating = row['rating']
        cost = row['actual_cost_usd']
        
        print(f"Generating evaluation report {idx + 1}/{len(highest_rated_answers)}: {question_model} → {answer_model}")
        
        # Add section separator
        markdown_sections.append(f"## 📋 Evaluation {idx + 1} of {len(highest_rated_answers)}")
        markdown_sections.append(f"**Question by:** {question_model} | **Answer by:** {answer_model} | **Avg Rating:** {avg_rating:.2f}/10 | **Cost:** ${cost:.4f}")
        markdown_sections.append("")
        
        # Generate the detailed evaluation markdown
        evaluation_md = generate_answer_evaluation_markdown(question_model, answer_model, ratings_df)
        markdown_sections.append(evaluation_md)
        
        # Add separator between evaluations
        markdown_sections.append("")
        markdown_sections.append("---")
        markdown_sections.append("")
    
    # Join all sections into final report
    final_report = "\n".join(markdown_sections)
    
    print(f"\n✓ Generated complete report with {len(highest_rated_answers)} detailed evaluations")
    
    return final_report

In [ ]:
from IPython.display import display, Markdown

# Generate the full markdown report
markdown_report = []
markdown_report.append("# 📋 Comprehensive Model Evaluation Report\n")
markdown_report.append(top_models_markdown(top_models) + "\n")
markdown_report.append(model_performance_summary_markdown(summary_model))
markdown_report.append(generate_cost_summary_markdown(cost_summary_df))
markdown_report.append(cross_model_rating_matrix_markdown(summary_pair))
markdown_report.append(generate_performance_tables_markdown(cost_summary_df))
markdown_report.append(cost_breakdown_by_phase_markdown(cost_summary_df))
markdown_report.append(most_expensive_calls_markdown(cost_summary_df))
markdown_report.append(generate_rating_statistics_markdown(ratings_df))
# markdown_report.append(questions_to_markdown(questions_df))
markdown_report.append(generate_highest_rated_answers_report(ratings_df_full, cost_summary_df))


final_markdown_report = "\n".join(markdown_report)

# Display the report in the notebook
display(Markdown(final_markdown_report))

In [ ]:
from datetime import datetime
from pathlib import Path

def save_markdown_report(report: str, folder: str = "results", prefix: str = "evaluation_report") -> Path:
    """
    Save a markdown string to the results folder with a timestamped filename.
    Returns the Path to the saved file.
    """
    # Ensure folder exists
    out_dir = Path(folder)
    out_dir.mkdir(parents=True, exist_ok=True)

    # Timestamp safe for filenames
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{prefix}_{ts}.md"
    out_path = out_dir / filename

    # Write file
    out_path.write_text(report, encoding="utf-8")
    print(f"✓ Saved markdown report to: {out_path}")
    return out_path

report_content = final_markdown_report  # from previous cell 41

# Save report
saved_path = save_markdown_report(report_content, folder="../results", prefix="evaluation_report")

## Interpretation & Next Steps

### What Do These Results Tell Us?

1. **Average Ratings** - Higher average ratings suggest models that consistently provide well-reasoned, complete answers
2. **Cross-Model Matrix** - Shows agreement/disagreement between raters. High variance may indicate:
   - Different evaluation standards between models
   - Genuine quality differences in answers
   - Bias toward certain answer styles

3. **Self-Ratings vs Peer-Ratings** - Reveals whether models are overconfident, overly critical, or fair

### Limitations to Consider

- **Sample Size**: Results are based on a small number of questions (N models = N questions)
- **Question Quality**: The evaluation quality depends on how good the generated questions are
- **Rater Bias**: Models may have inherent biases in how they evaluate responses
- **Domain Coverage**: Questions may cluster in certain domains based on model training

### Ideas for Extension

1. **Increase Sample Size**: Run with more questions per model (requires API credits)
2. **Domain-Specific Evaluation**: Test models on specific domains (math, coding, creative writing)
3. **Human Validation**: Compare model ratings with human expert ratings
4. **Consistency Testing**: Run the same evaluation multiple times to check stability
5. **Cost Analysis**: Track token usage and costs to compute value-per-dollar
6. **Response Time Analysis**: Compare speed vs quality tradeoffs
7. **Temperature Experiments**: Test how different temperature settings affect question/answer quality

### Saving Results

You can export the results for further analysis:

```python
# Export to CSV
summary_model_full.to_csv('model_ratings.csv', index=False)
summary_pair_full.to_csv('cross_model_ratings.csv', index=False)
questions_df_full.to_csv('generated_questions.csv', index=False)
answers_df_full.to_csv('model_answers.csv', index=False)
ratings_df_full.to_csv('all_ratings.csv', index=False)
```

### Token Usage & Cost Tracking

The notebook now captures detailed token usage and cost information for every API call:

**What's Tracked:**
- **Input tokens**: Tokens sent to the model (prompt + context)
- **Output tokens**: Tokens generated by the model (response)
- **Total tokens**: Input + output tokens
- **Estimated cost**: Based on approximate OpenRouter pricing

**Cost Calculation:**
- Input tokens: ~$3 per 1M tokens (rough average)
- Output tokens: ~$15 per 1M tokens (rough average)
- Actual rates vary significantly by model and provider

**Export Enhanced Data:**
```python
# Export all data with token/cost tracking
questions_df_full.to_csv('questions_with_costs.csv', index=False)
answers_df_full.to_csv('answers_with_costs.csv', index=False) 
ratings_df_full.to_csv('ratings_with_costs.csv', index=False)
cost_summary_df.to_csv('cost_summary.csv', index=False)
```

**Benefits:**
- Track experiment costs in real-time
- Compare cost efficiency across models
- Budget for larger evaluations
- Identify expensive operations (long answers vs short ratings)

## Run Full Evaluation Pipeline

Execute the complete pipeline: question generation → answering → rating

In [ ]:
# Run the complete evaluation pipeline with cost tracking
# NOTE: Make sure to execute all function definition cells above before running this cell

print(f"Starting evaluation pipeline with {len(top_models)} models...\n")

# Step 1: Generate questions
questions_df_full, valid_questions_df_full = generate_questions(top_models, client)

# Step 2: Generate answers
answers_df_full = answer_questions(valid_questions_df_full, top_models, client)

# Step 3: Evaluate answers
ratings_df_full = evaluate_answers(answers_df_full, top_models, client)

# Step 4: Generate comprehensive cost and token summary
cost_summary_df = generate_cost_summary(questions_df_full, answers_df_full, ratings_df_full)

print("\n" + "=" * 80)
print("PIPELINE COMPLETE")
print("=" * 80)
print("\nGenerated artifacts:")
print("  - questions_df_full: All generated questions (with token/cost tracking)")
print("  - valid_questions_df_full: Successfully generated questions only")
print("  - answers_df_full: All model answers to all questions (with token/cost tracking)")
print("  - ratings_df_full: All ratings from all rater models (with token/cost tracking)")
print("  - summary_model_full: Average rating per model")
print("  - summary_pair_full: Cross-model rating matrix")
print("  - cost_summary_df: Comprehensive token usage and cost breakdown")

### 🧪 Quick Test (Optional)

Before running the full evaluation, you can test with a single model to verify everything works:

```python
# Test with just one model
test_model = top_models.head(1)
print(f"Testing with: {test_model.iloc[0]['model_name']}")

# Generate one question
test_q, test_valid_q = generate_questions(test_model, client)
if len(test_valid_q) > 0:
    print("✓ Question generation works!")
    
# Generate one answer
test_a = answer_questions(test_valid_q, test_model, client)
if len(test_a) > 0:
    print("✓ Answer generation works!")
        
# Generate one rating
test_r, _, _ = evaluate_answers(test_a, test_model, client)
if len(test_r) > 0:
    print("✓ Rating works!")
    print("\n🎉 All systems go! Ready for full evaluation.")
```

**Tip:** Run this test first if you're unsure about your API setup.

In [ ]:

ratings_df_full.head()



# Charts and Graphs
Here are some charts that analyze the model performance based on the ratings_df_full dataframe.
## Average Rating per Model
This chart shows the average rating for each model, providing a clear comparison of their overall performance.


In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Calculate average rating per model
avg_rating_per_model = ratings_df_full.groupby('answer_model_name')['rating'].mean().sort_values(ascending=False).reset_index()

# Calculate average cost per model
avg_cost_per_model = ratings_df_full.groupby('answer_model_name')['actual_cost_usd'].mean().reset_index()

# Merge the two dataframes
merged_df = pd.merge(avg_rating_per_model, avg_cost_per_model, on='answer_model_name')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=merged_df['answer_model_name'], y=merged_df['rating'], name='Average Rating'),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=merged_df['answer_model_name'], y=merged_df['actual_cost_usd'], name='Average Cost (USD)', mode='lines+markers'),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text='Average Rating and Cost per Model'
)

# Set x-axis title
fig.update_xaxes(title_text='Model')

# Set y-axes titles
fig.update_yaxes(title_text='Average Rating', secondary_y=False)
fig.update_yaxes(title_text='Average Cost (USD)', secondary_y=True)


fig.show()


## Rating Distribution per Model
This box plot shows the distribution of ratings for each model. It helps to visualize the consistency and spread of ratings for each model.


In [ ]:

fig = px.box(ratings_df_full,
             x='answer_model_name',
             y='rating',
             title='Rating Distribution per Model',
             labels={'rating': 'Rating', 'answer_model_name': 'Model'},
             color='answer_model_name')
fig.show()



## Cost vs. Rating
This scatter plot shows the relationship between the actual cost in USD and the rating for each model. This can help identify models that are both high-performing and cost-effective.


In [ ]:

fig = px.scatter(ratings_df_full,
                 x='actual_cost_usd',
                 y='rating',
                 color='answer_model_name',
                 title='Cost vs. Rating',
                 labels={'actual_cost_usd': 'Actual Cost (USD)', 'rating': 'Rating'},
                 hover_data=['answer_model_name'])
fig.show()
